In [5]:
import time
from klampt import *
from klampt.math import vectorops,so3,se3
import math
from IPython.display import clear_output
import ipywidgets as widgets
import ipyklampt

world = WorldModel()
fn = "data/ur5Blocks.xml"
res = world.readFile(fn)
robot = world.robot(0)

#set the home configuration
qhome  = robot.getConfig()
qhome[1] = 2.3562
qhome[2] = -math.pi/3.0
qhome[3] = math.pi*2/3.0
qhome[4] = 0.64181
qhome[5] = math.pi/2.0
robot.setConfig(qhome)

kvis = ipyklampt.KlamptWidget(world,width=600,height=400)
kvis.set_camera({u'near': 0.1, u'target': {u'y': 1.1188322004142854, u'x': 0.042176605196346695, u'z': 0.009329657831685366}, u'far': 1000, u'position': {u'y': 1.6142988723996625, u'x': 1.5814619610767169, u'z': -0.03643442712963929}, u'up': {u'y': 1, u'x': 0, u'z': 0}})

#If you'd like to show the print output, comment out this line
#clear_output()
display(kvis)
controls = widgets.HBox([])
display(controls)

#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

S2xhbXB0V2lkZ2V0KGNhbWVyYT17dSdmYXInOiAxMDAwLCB1J3Bvc2l0aW9uJzoge3UneSc6IDEuNjE0Mjk4ODcyMzk5NjYyNSwgdSd4JzogMS41ODE0NjE5NjEwNzY3MTY5LCB1J3onOiAtMC7igKY=


HBox()

In [6]:
from klampt.model import ik

#information about the end effector
ee_link = 7
#the driver responsible for changing the gripper value
ee_driver = 6
#the position of the center link
ee_localpos = (0.15,0,0)
#the direction pointing from the palm toward the fingers
ee_fwd_dir = (1,0,0)
#the direction pointing perpendicular to the fingers
ee_perpendicular_dir = (0,0,1)

#driver parameters
ee_driver_open_value = 0.0
ee_driver_closed_value = 0.723

def set_gripper_width(robot,q,width):
    """Helper: produces a configuration which is the same as q, except that the
    gripper is opened to the given width (in the range 0 (closed) to 1 (open))"""
    qold = robot.getConfig()
    robot.setConfig(q)
    robot.driver(ee_driver).setValue(ee_driver_open_value*width + (1.0-width)*ee_driver_closed_value)
    qres = robot.getConfig()
    robot.setConfig(qold)
    return qres

def find_pregrasp_config(robot,block_pose,offset_height=0.06):
    """
    TODO: complete me for C.1
    
    In:
    - robot: a RobotModel instance, with the robot currently posed at qhome
    - block_pose: a transform of the center of the block
    - offset_height: the distance to hover above the block
    
    Out: a configuration q reaching the target pregrasp.
      None can be returned if there is no solution, but in all test cases there should be a solution.
    """
    return set_gripper_width(robot,robot.getConfig(),1)

def lower_and_close(robot,block_pose,close_width):
    """
    TODO: complete me for C.2
    
    In:
    - robot: a RobotModel instance, with the robot currently posed at the pregrasp configuration
    - block_pose: a transform of the center of the block
    - close_width: the amount that the gripper should be closed
    
    Out: a list of milestones interpolating from the robot's current configuration
    moving straight down (in Cartesian space), and then closing the robot's gripper.
    
    You may return None if you cannot find a solution, but for the test cases you should
    always be able to find a solution.
    """
    qstart = robot.getConfig()
    qdown = qstart[:]
    qclose = set_gripper_width(robot,qdown,close_width)
    path = []
    path.append(qstart)
    path.append(qdown)
    path.append(qclose)
    return path
    
def lift_and_place(robot,target_position):
    """
    TODO: complete me for C.3
    
    In:
    - robot: a RobotModel instance, with the robot currently gripping an object
    - target_position: the position of the target box
    
    Out: a list of milestones interpolating from the robot's current configuration
    moving 1) to lift the block up, 2) to the target position, and 3) opening the gripper
    
    You may return None if you cannot find a solution, but for the test cases you should
    always be able to find a solution.
    """
    qstart = robot.getConfig()
    qup = qstart[:]
    qbox = qup[:]
    qopen = set_gripper_width(robot,qbox,1)
    return [qstart,qup,qbox,qopen]

In [7]:
#run this cell to bind your code to the visualization playback window. You should 
#not need to modify this code, unless you want to tweak the trajectory execution speed

#target parameters
target_position = (0.26,-0.15,1.2)

#block parameters
block_width = 0.05
block_names = [world.rigidObject(i).getName() for i in range(world.numRigidObjects())]
block_poses = [world.rigidObject(i).getTransform() for i in range(world.numRigidObjects())]

#gripper parameters
block_close_amount = 0.5

#the speed of trajectory execution: default 0.5 radian per second
#this may be tweaked, if desired
trajectory_execution_speed = 0.5

from klampt.model.trajectory import RobotTrajectory

#state can be 'HOME', 'PREGRASP', 'GRASPED', 'BOX'
state = 'HOME'
grasped_objects = []
grasp_xforms = []
block_grasp_threshold = 0.55
objectTableHeight = 0.90

#the amount of time between refreshes
dt = 0.05

kvis.add_text("HUD1",1,1)
kvis.add_text("HUD2",1,5)
kvis.add_text("HUD3",1,9)
kvis.add_text("HUD4",1,13)
kvis.add_text("HUD5",1,17)
kvis.add_text("HUD6",1,21)
kvis.add_text("HUD7",1,25)

def update_grasp_logic():
    global robot,ee_driver,grasped_objects,grasp_xforms
    width = (robot.driver(ee_driver).getValue() - ee_driver_closed_value)/(ee_driver_open_value-ee_driver_closed_value)
    held = (width <= block_grasp_threshold)
    if not held:
        #drop everything
        for o in grasped_objects:
            T = world.rigidObject(o).getTransform()
            T[1][2] = objectTableHeight
            world.rigidObject(o).setTransform(*T)
        grasped_objects = []
        grasp_xforms = []
    else:
        #update object transforms from robot movement
        Tgripper = robot.link(ee_link).getTransform()
        gripper_world_pos = se3.apply(Tgripper,ee_localpos)
        for o,Tgrasp in zip(grasped_objects,grasp_xforms):
            world.rigidObject(o).setTransform(*se3.mul(Tgripper,Tgrasp))
        for o in range(world.numRigidObjects()):
            if o in grasped_objects:
                continue
            if vectorops.distance(gripper_world_pos,world.rigidObject(o).getTransform()[1]) <= block_width*0.5:
                grasped_objects.append(o)
                Tgrasp = se3.mul(se3.inv(Tgripper),world.rigidObject(o).getTransform())
                grasp_xforms.append(Tgrasp)

def execute_trajectory(traj):
    global robot,controls
    for c in controls.children:
        c.disabled = True
    t = traj.times[0]
    while t < traj.times[-1]:
        q = traj.eval(t)
        robot.setConfig(q)
        update_grasp_logic()
        kvis.update()
        time.sleep(dt)
        t += dt
    q = traj.milestones[-1]
    robot.setConfig(q)
    kvis.update()
    for c in controls.children:
        c.disabled = False

def trajectory_from_path(path):
    global robot
    times = [0.0]
    for i in range(1,len(path)):
        d = robot.distance(path[i-1],path[i])
        times.append(times[-1]+d/trajectory_execution_speed)
    print "Inferred path duration",times[-1]-times[0]
    return RobotTrajectory(robot,times,path)

def execute_path(path):
    execute_trajectory(trajectory_from_path(path))
    
def do_move_home(button):
    global state,robot,qhome
    execute_path([robot.getConfig(),qhome])
    state = 'HOME'

def do_move_pregrasp(button):
    global state,robot
    if state != 'HOME':
        print "WARNING: should do pregrasp from the home configuration"
    qstart = robot.getConfig()
    selected_object = object_dropdown.value
    qtgt = find_pregrasp_config(robot,world.rigidObject(selected_object).getTransform())
    if qtgt is not None:
        execute_path([qstart,qtgt])
        state = 'PREGRASP'

def do_grasp(button):
    global state,robot
    if state != 'PREGRASP':
        print "WARNING: should do grasp from the pregrasp configuration"
    selected_object = object_dropdown.value
    milestones = lower_and_close(robot,world.rigidObject(selected_object).getTransform(),block_close_amount)
    if milestones is not None:
        execute_path(milestones)
        state = 'GRASPED'

def do_place(button):
    global state,robot
    if state != 'GRASPED':
        print "WARNING: should do place from the grasped configuration"
    milestones = lift_and_place(robot,target_position)
    if milestones is not None:
        execute_path(milestones)
        state = 'BOX'
        
object_dropdown = widgets.Dropdown(
    options=block_names,
    value=block_names[0],
    description='Object:'
)

move_home_button = widgets.Button(description="Move Home")
move_pregrasp_button = widgets.Button(description="Move to Pregrasp")
grasp_button = widgets.Button(description="Grasp")
place_button = widgets.Button(description="Place")
move_home_button.on_click(do_move_home)
move_pregrasp_button.on_click(do_move_pregrasp)
grasp_button.on_click(do_grasp)
place_button.on_click(do_place)

controls.children = (move_home_button,object_dropdown,move_pregrasp_button,grasp_button,place_button)

In [4]:
#Test movement to home
qtest = qhome[:]
qtest[1] -= 2.0
robot.setConfig(qtest)
kvis.update()